In [1]:
import pandas as pd
data = pd.read_csv("churn_modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [2]:
x = data.iloc[:, 3:13]
y = data.iloc[:, 13]
geography = pd.get_dummies(x["Geography"], drop_first = True)
gender = pd.get_dummies(x["Gender"], drop_first = True)
x = pd.concat([x, geography, gender], axis = 1)
x = x.drop(["Geography", "Gender"], axis = 1)
x

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [4]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

In [5]:
data.shape[1]

14

In [6]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(units = nodes, input_dim = x_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
        else:
            model.add(Dense(units = nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'glorot_uniform'))
    
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    return model

In [9]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

layers = [[40, 20], [20], [45, 30, 15]]
activations = ["sigmoid", "relu"]
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = 5)

grid_result = grid.fit(x_train, y_train)

<ipython-input-9-0bd40039f496>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = create_model, verbose = 0)


In [17]:
grid_result.best_score_, grid_result.best_params_

(0.793874990940094,
 {'activation': 'sigmoid', 'batch_size': 128, 'epochs': 30, 'layers': [20]})

In [18]:
neuron_model = Sequential()
neuron_model.add(Dense(units = 20, activation = "sigmoid", kernel_initializer = "glorot_uniform", input_dim = x_train.shape[1]))
neuron_model.add(Dense(units = 20, activation = "sigmoid", kernel_initializer = "glorot_uniform"))
neuron_model.add(Dense(units = 20, activation = "sigmoid", kernel_initializer = "glorot_uniform"))
neuron_model.add(Dense(units = 1, activation = "sigmoid" , kernel_initializer = "glorot_uniform"))
neuron_model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
neuron_model.fit(x_train, y_train, batch_size = 100, epochs = 30)

Epoch 1/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5931 - accuracy: 0.6883
Epoch 2/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5048 - accuracy: 0.7969
Epoch 3/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5040 - accuracy: 0.7969
Epoch 4/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5036 - accuracy: 0.7969
Epoch 5/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5027 - accuracy: 0.7969
Epoch 6/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5021 - accuracy: 0.7969
Epoch 7/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5013 - accuracy: 0.7969
Epoch 8/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5012 - accuracy: 0.7969
Epoch 9/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5010 - accuracy: 0.7969
Epoch 10/30
80/80 [==============================] - 0s 1ms/step - loss: 0.5009 - accuracy: 0.7969
Epoch 11/30
80/80 [

In [20]:
y_pred = neuron_model.predict(x_test)
y_pred = (y_pred>0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [21]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)
score

0.794